In [1]:
import cv2, numpy as np, os

In [2]:
# Video & reference
video = "C:\\Users\\Darshan\\Downloads\\Computer Vision\\mpeg4.mp4"
ref = cv2.imread("C:\\Users\\Darshan\\Downloads\\Computer Vision\\VideoShot.jpg", 0)

In [3]:
# SIFT setup
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(ref, None)
match = cv2.BFMatcher()

In [4]:
cap = cv2.VideoCapture(video)
print("Scanning...")

Scanning...


In [5]:
frame, hits = 0, 0
while cap.isOpened():
    ret, f = cap.read()
    if not ret:break
    frame += 1

    gray = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
    kp2, des2 = sift.detectAndCompute(gray, None)
    if des2 is None: continue

    good = [m for m, n in match.knnMatch(des1, des2, 2) if m.distance < 0.75 * n.distance]

    if len(good) >= 10:
        src = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

        if mask is not None:
            inl = mask.ravel().tolist()
            percent = sum(inl) / len(good) * 100

            if percent > 50:
                hits += 1
                print(f" Frame {frame}: {percent:.1f}% match")
                img = cv2.drawMatches(ref, kp1, gray, kp2, good, None, matchesMask=inl, flags=2)
                cv2.imshow("Image",img)
                key = cv2.waitKey(0)  # Wait for any key to go to next frame
                if key == 27:  # Press Esc to break early
                    break

cap.release()
cv2.destroyAllWindows()
print(f"\n Done. Detected in {hits} frame(s).")

 Frame 6: 61.1% match
 Frame 13: 61.5% match
 Frame 15: 54.5% match
 Frame 16: 70.6% match
 Frame 21: 54.5% match
 Frame 22: 63.6% match
 Frame 23: 66.7% match
 Frame 24: 71.4% match
 Frame 25: 58.3% match
 Frame 26: 72.0% match
 Frame 30: 55.6% match

 Done. Detected in 11 frame(s).
